In [2]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #
import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm


beta_rel=0.09423258405
gamma = 1.004469679

current=0.6 #A
cooler_length = 3 # m cooler length
r_beam=25*1e-3 #m

T_perp = 0.01 # <E> [eV] = kb*T
T_l =  0.001 # <E> [eV]
magnetic_field = 0.07 # T for LEIR
B_ratio=0

mass0=193729.0248722061*1e6 #eV/c2
c=299792458.0
p0c = mass0*beta_rel*gamma #eV/c

circumference = 78.54370266 #m
T = circumference/(c*beta_rel)
ms_per_turn = 1000*T
s_per_turn = T

beta_x=5
beta_y=5


disp_x=0
Q_x=1.82
Q_y=2.72
dQx=-0.0005113579791
dQy=-0.0001665178543

arc = xt.LineSegmentMap(
        qx=Q_x, qy=Q_y, qs=0,
        dqx=dQx, dqy=dQy,
        bets = 1*1e40, length=circumference,
        betx=beta_x,
        bety=beta_y
        )
num_particles=int(1e6)

#force againt velocity
T_perp_list = [0.001,0.01,0.1] # eV

for T_perp in tqdm(T_perp_list):

    emittance=14*1e-6 #inital emittance
    px=np.random.normal(0, 4*np.sqrt(emittance/beta_x), num_particles)
    dtk_particle = dtk.TestParticles(
        
        mass0=mass0,
        p0c=p0c,
        x=np.random.normal(0,1e-20 ,num_particles),
        px=px,
        y=np.random.normal(0, 1e-20 ,num_particles),
        py=np.random.normal(0, 1e-20 ,num_particles),
        delta=np.random.normal(0, 1e-20 ,num_particles),
        zeta=np.random.normal(0, 0 ,num_particles),
        q0=54)

    dtk_cooler = dtk.elements.ElectronCooler(current=current, length=cooler_length, radius_e_beam=r_beam,
                            temp_perp=T_perp, temp_long=T_l,
                            magnetic_field=magnetic_field, magnetic_field_ratio=B_ratio,
                            neutralisation_space_charge=0)
    force,Fy,Fl = dtk_cooler.force(dtk_particle)

    def temperature_to_velocity_1D(temperature_eV):
        
        T=temperature_eV#J
        mass = 511 *1e3
        velocity = c*np.sqrt( T/ mass)
        return velocity

    px_tot = p0c*dtk_particle.px
    beta_diff = px_tot/(mass0*gamma)
    v_diff = beta_diff*c
    # Sort v_diff and force in ascending order
    sort_indices = np.argsort(v_diff)
    v_diff = v_diff[sort_indices]
    force = force[sort_indices]
   
    filepath=f'results/force_xsuite_T_perp={T_perp:.0e}.npz'
    np.savez(filepath, v_diff=v_diff, force=force)

100%|██████████| 3/3 [00:36<00:00, 12.05s/it]
